In [2]:
import gymnasium as gym

from envs.rllib import DummyEnvironment
from src.wrappers.rllib import UniformlyRestrictedDiscreteEnvironment
from src.utils import test_environment

ModuleNotFoundError: No module named 'src'

In [ ]:
test_environment(DummyEnvironment)

In [ ]:
env_config = { 'action_space': gym.spaces.Discrete(3) }

test_environment(UniformlyRestrictedDiscreteEnvironment({ 'env': DummyEnvironment, 'env_config': env_config }))